In [1]:
!pip install numpy pandas scanpy mygene openai biopython html2text torchcfm

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 42.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.1/375.1 kB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 138.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.0/129.0 kB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import os
import pickle
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchdyn.core import NeuralODE
from tqdm import tqdm
import scanpy as sc
from torchcfm.conditional_flow_matching import ConditionalFlowMatcher
from torchcfm.models import MLP
import pickle
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchdyn.core import NeuralODE
from tqdm import tqdm
import scanpy as sc
from torchcfm.conditional_flow_matching import ConditionalFlowMatcher
import torch.optim as optim
import torch.distributed as dist
import torch.multiprocessing as mp

import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data import DataLoader, DistributedSampler
from torchdyn.core import NeuralODE
from tqdm import tqdm


import numpy as np
from absl import app, flags

import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder

import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Subset

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load AnnData object
adata = sc.read_h5ad('/home/ubuntu/D2_all_clean_hvg_filtered.h5ad')

# Load GenePT embeddings
pickle_file = "/home/ubuntu/GenePT_gene_embedding_ada_text_updated2.pickle"

with open(pickle_file, 'rb') as fp:
    gene_embeddings = pickle.load(fp)



/home/ubuntu/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
# Configuration
config = {
    "lr": 1e-4,
    "batch_size": 1024,
    "num_epochs": 10,
    "save_dir": "/home/ubuntu/gene_expression_flow_matching",
    "parallel": True
}

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

In [3]:
# Function to get embedding for a gene or average of multiple genes
def get_embedding(gene_names):
    genes = gene_names.split('+')
    embeddings = [gene_embeddings[gene] for gene in genes if gene in gene_embeddings]
    return np.mean(embeddings, axis=0) if embeddings else np.zeros(1536)



def prepare_data(adata, val_ratio=0.02):
    datasets = []
    organisms = []
    cell_types = []
    total_samples = 0

    organism_encoder = LabelEncoder()
    cell_type_encoder = LabelEncoder()

    organism_encoder.fit(adata.obs['organism'])
    cell_type_encoder.fit(adata.obs['cell_type'])

    dataset_identifiers = adata.obs.index.str.extract(r'(dataset_\d+_[a-f0-9]+)')[0].unique()

    for dataset_id in dataset_identifiers:
        dataset_adata = adata[adata.obs.index.str.startswith(dataset_id)]
        
        print(f"\nProcessing Dataset {dataset_id}")
        print(f"Total samples in dataset: {dataset_adata.shape[0]}")
        
        control_data = dataset_adata[dataset_adata.obs['condition'] == 'control'].X.toarray()
        test_data = dataset_adata[dataset_adata.obs['condition'] == 'test'].X.toarray()
        
        print(f"Initial Control samples: {control_data.shape[0]}, Test samples: {test_data.shape[0]}")

        if control_data.shape[0] > 0 and test_data.shape[0] > 0:
            if control_data.shape[0] >= test_data.shape[0]:
                control_indices = np.random.choice(control_data.shape[0], test_data.shape[0], replace=False)
                control_data = control_data[control_indices]
            else:
                repeat_factor = int(np.ceil(test_data.shape[0] / control_data.shape[0]))
                control_data = np.tile(control_data, (repeat_factor, 1))[:test_data.shape[0]]
            
            samples_count = test_data.shape[0]
            perturbations = dataset_adata[dataset_adata.obs['condition'] == 'test'].obs['perturbation_name'].tolist()
        elif control_data.shape[0] > 0:
            test_data = control_data.copy()
            samples_count = control_data.shape[0]
            perturbations = ['no_perturbation'] * samples_count
        elif test_data.shape[0] > 0:
            control_data = test_data.copy()
            samples_count = test_data.shape[0]
            perturbations = dataset_adata[dataset_adata.obs['condition'] == 'test'].obs['perturbation_name'].tolist()
        else:
            print(f"Skipping dataset {dataset_id} - no samples")
            continue

        print(f"Final Control samples: {control_data.shape[0]}, Test samples: {test_data.shape[0]}")
        
        total_samples += samples_count
        
        perturbation_embeddings = np.array([get_embedding(p) for p in perturbations[:samples_count]])
        
        final_organisms = organism_encoder.transform(dataset_adata.obs['organism'].values[:samples_count])
        final_cell_types = cell_type_encoder.transform(dataset_adata.obs['cell_type'].values[:samples_count])
        
        datasets.append((control_data, test_data, perturbation_embeddings))
        organisms.append(final_organisms)
        cell_types.append(final_cell_types)

    print(f"\nTotal samples across all datasets: {total_samples}")

    # Combine all datasets
    all_control = np.vstack([d[0] for d in datasets])
    all_test = np.vstack([d[1] for d in datasets])
    all_embeddings = np.vstack([d[2] for d in datasets])
    all_organisms = np.concatenate(organisms)
    all_cell_types = np.concatenate(cell_types)

    print(f"Shape of combined control data: {all_control.shape}")
    print(f"Shape of combined test data: {all_test.shape}")
    print(f"Shape of combined embeddings: {all_embeddings.shape}")

    # Create train-validation split
    train_indices, val_indices = train_test_split(
        np.arange(total_samples), 
        test_size=val_ratio, 
        stratify=all_organisms  # Stratify by organism to ensure balanced split
    )

    return (
        (all_control, all_test, all_embeddings, all_organisms, all_cell_types),
        train_indices,
        val_indices,
        organism_encoder,
        cell_type_encoder
    )

# # Usage
# all_data, train_indices, val_indices, organism_encoder, cell_type_encoder = prepare_data(adata)

# Usage:


class GeneExpressionDataset(Dataset):
    def __init__(self, control_data, test_data, embeddings, organisms, cell_types):
        self.control_data = control_data
        self.test_data = test_data
        self.embeddings = embeddings
        self.organisms = organisms
        self.cell_types = cell_types

        assert len(control_data) == len(test_data) == len(embeddings) == len(organisms) == len(cell_types), "All input arrays must have the same length"

    def __len__(self):
        return len(self.control_data)

    def __getitem__(self, idx):
        return (
            torch.FloatTensor(self.control_data[idx]),
            torch.FloatTensor(self.test_data[idx]),
            torch.FloatTensor(self.embeddings[idx]),
            torch.tensor(self.organisms[idx], dtype=torch.long),
            torch.tensor(self.cell_types[idx], dtype=torch.long)
        )



# full_dataset = GeneExpressionDataset(*all_data)
# train_dataset = Subset(full_dataset, train_indices)
# val_dataset = Subset(full_dataset, val_indices)

# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)

# print(f"Training samples: {len(train_dataset)}")
# print(f"Validation samples: {len(val_dataset)}")


# Usage:

class ODEFunc(nn.Module):
    def __init__(self, model, embedding):
        super().__init__()
        self.model = model
        self.embedding = embedding

    def forward(self, t, x, *args, **kwargs):
        t = t.reshape(-1).repeat(x.shape[0])
        return self.model(x, t, self.embedding)



def calculate_mse_by_group(pred, true, groups):
    mse_fn = nn.MSELoss(reduction='none')
    mse = mse_fn(pred, true).mean(dim=1)  # MSE for each sample

    group_mse = {}
    for group in torch.unique(groups):
        mask = groups == group
        group_mse[group.item()] = mse[mask].mean().item()

    return group_mse


# Define the CustomMLP
class CustomMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers):
        super().__init__()
        self.layers = nn.ModuleList()
        self.layers.append(nn.Linear(input_dim, hidden_dim))
        for _ in range(num_layers - 2):
            self.layers.append(nn.Linear(hidden_dim, hidden_dim))
        self.layers.append(nn.Linear(hidden_dim, output_dim))
        self.activation = nn.ReLU()

    def forward(self, x):
        for layer in self.layers[:-1]:
            x = self.activation(layer(x))
        return self.layers[-1](x)

# Define the GeneExpressionFlowMatcher model
class GeneExpressionFlowMatcher(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim=256, num_layers=5):
        super().__init__()
        self.model = CustomMLP(input_dim + embedding_dim + 1, hidden_dim, input_dim, num_layers)

    def forward(self, x, t, embedding):
        return self.model(torch.cat([x, embedding, t.unsqueeze(1)], dim=1))

# # Initialize model, optimizer, and flow matcher
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# input_dim = all_data[0].shape[1]  # Use the shape of the combined control data
# embedding_dim = 1536
# model = GeneExpressionFlowMatcher(input_dim, embedding_dim).to(device)
# optimizer = optim.Adam(model.parameters(), lr=1e-4)
# FM = ConditionalFlowMatcher(sigma=0.1)

# # Print model details
# print(f"Input dimension: {input_dim}")
# print(f"Embedding dimension: {embedding_dim}")
# print(f"Model architecture:\n{model}")


In [4]:
def train():
    # Data preparation
    all_data, train_indices, val_indices, organism_encoder, cell_type_encoder = prepare_data(adata)
    full_dataset = GeneExpressionDataset(*all_data)
    train_dataset = Subset(full_dataset, train_indices)
    val_dataset = Subset(full_dataset, val_indices)

    train_loader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=config["batch_size"], shuffle=False)

    print(f"Training samples: {len(train_dataset)}")
    print(f"Validation samples: {len(val_dataset)}")

    # Model initialization
    input_dim = all_data[0].shape[1]
    embedding_dim = 1536
    model = GeneExpressionFlowMatcher(input_dim, embedding_dim).to(device)
    
    if config["parallel"] and torch.cuda.device_count() > 1:
        print(f"Using {torch.cuda.device_count()} GPUs!")
        model = nn.DataParallel(model)
    
    optimizer = optim.Adam(model.parameters(), lr=config["lr"])
    FM = ConditionalFlowMatcher(sigma=0.1)

    print(f"Input dimension: {input_dim}")
    print(f"Embedding dimension: {embedding_dim}")
    print(f"Model architecture:\n{model}")

    # Training loop
    for epoch in range(config["num_epochs"]):
        model.train()
        total_loss = 0

        for x0, x1, embedding, organism, cell_type in tqdm(train_loader, desc=f'Epoch {epoch+1}/{config["num_epochs"]}'):
            x0, x1, embedding = x0.to(device), x1.to(device), embedding.to(device)

            optimizer.zero_grad()

            t, xt, ut = FM.sample_location_and_conditional_flow(x0, x1)
            vt = model(xt, t, embedding)

            loss = torch.mean((vt - ut) ** 2)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        print(f'Epoch {epoch+1}/{config["num_epochs"]}, Average Training Loss: {avg_loss:.4f}')

        # Validation
        model.eval()
        val_loss = 0
        all_pred = []
        all_true = []
        all_organisms = []
        all_cell_types = []

        with torch.no_grad():
            for x0, x1, embedding, organism, cell_type in val_loader:
                x0, x1, embedding = x0.to(device), x1.to(device), embedding.to(device)
                organism, cell_type = organism.to(device), cell_type.to(device)

                ode_func = ODEFunc(model.module if config["parallel"] else model, embedding)
                node = NeuralODE(ode_func, sensitivity='adjoint', solver='dopri5', atol=1e-4, rtol=1e-4).to(device)
                trajectory = node.trajectory(x0, t_span=torch.linspace(0, 1, 2).to(device))
                pred_x1 = trajectory[-1]

                loss = torch.mean((pred_x1 - x1) ** 2)
                val_loss += loss.item()

                all_pred.append(pred_x1)
                all_true.append(x1)
                all_organisms.append(organism)
                all_cell_types.append(cell_type)

        all_pred = torch.cat(all_pred, dim=0)
        all_true = torch.cat(all_true, dim=0)
        all_organisms = torch.cat(all_organisms, dim=0)
        all_cell_types = torch.cat(all_cell_types, dim=0)

        # Overall MSE
        overall_mse = nn.MSELoss()(all_pred, all_true)
        print(f'Overall Validation MSE: {overall_mse.item():.4f}')

        # MSE by organism and cell type
        organism_mse = calculate_mse_by_group(all_pred, all_true, all_organisms)
        cell_type_mse = calculate_mse_by_group(all_pred, all_true, all_cell_types)

        print("MSE by Organism:")
        for org, mse in organism_mse.items():
            org_name = organism_encoder.inverse_transform([org])[0]
            print(f"  {org_name}: {mse:.4f}")

        print("MSE by Cell Type:")
        for ct, mse in cell_type_mse.items():
            ct_name = cell_type_encoder.inverse_transform([ct])[0]
            print(f"  {ct_name}: {mse:.4f}")

        # Save the model
        torch.save(model.state_dict(), f'{config["save_dir"]}/model_epoch_{epoch+1}.pt')

    print("Training completed!")

if __name__ == "__main__":
    train()


Processing Dataset dataset_0_d5493bc6
Total samples in dataset: 23608
Initial Control samples: 2446, Test samples: 21162
Final Control samples: 21162, Test samples: 21162

Processing Dataset dataset_1_43ff3fc2
Total samples in dataset: 29858
Initial Control samples: 2327, Test samples: 27531
Final Control samples: 27531, Test samples: 27531

Processing Dataset dataset_2_c9127ee6
Total samples in dataset: 45734
Initial Control samples: 45734, Test samples: 0
Final Control samples: 45734, Test samples: 45734

Processing Dataset dataset_3_87db8d3b
Total samples in dataset: 301112
Initial Control samples: 26344, Test samples: 274768
Final Control samples: 274768, Test samples: 274768

Processing Dataset dataset_4_09c14839
Total samples in dataset: 42669
Initial Control samples: 3423, Test samples: 39246
Final Control samples: 39246, Test samples: 39246

Processing Dataset dataset_5_b6e36a6e
Total samples in dataset: 408278
Initial Control samples: 16989, Test samples: 391289
Final Control

Epoch 1/10: 100%|██████████| 4367/4367 [06:31<00:00, 11.15it/s]


Epoch 1/10, Average Training Loss: 0.0067
Overall Validation MSE: 0.0072
MSE by Organism:
  Human Cytomegalovirus (HCMV): 0.0051
  Humans (Homo sapiens): 0.0081
  Mice (Mus musculus): 0.0022
  Zebrafish (Danio rerio): 0.0000
  human adipocytes: 0.0098
MSE by Cell Type:
  A172: 0.0227
  B-cell acute lymphoblastic leukemia (B-ALL) cell: 0.0000
  CAR T cell: 0.0044
  Endothelial Cells: 0.0042
  Hep G2: 0.0000
  Human Preadipocyte SGBS Cells, Adipocytes: 0.0098
  Human iPSCs, Human NSCs: 0.0033
  Jurkat: 0.0000
  K562 chronic myelogenous leukemia cell: 0.0025
  T Cytotoxic: 0.0043
  T Low quality cells: 0.0040
  T Memory: 0.0047
  T Naive: 0.0046
  T Th1/Cytotoxic: 0.0047
  T Th1/Th17: 0.0049
  T Th17: 0.0049
  T Th2: 0.0046
  T Treg memory 1: 0.0050
  T Treg memory 2: 0.0048
  T Treg memory 3: 0.0040
  T Treg naive: 0.0045
  T98G: 0.0223
  THP-1 cell: 0.0046
  U87MG: 0.0222
  astrocytes: 0.0043
  cortical organoid: 0.0069
  disseminated tumour cell: 0.0000
  embryonic kidney cell: 0.0032


Epoch 2/10: 100%|██████████| 4367/4367 [06:23<00:00, 11.38it/s]


Epoch 2/10, Average Training Loss: 0.0064
Overall Validation MSE: 0.0070
MSE by Organism:
  Human Cytomegalovirus (HCMV): 0.0051
  Humans (Homo sapiens): 0.0079
  Mice (Mus musculus): 0.0022
  Zebrafish (Danio rerio): 0.0000
  human adipocytes: 0.0096
MSE by Cell Type:
  A172: 0.0220
  B-cell acute lymphoblastic leukemia (B-ALL) cell: 0.0000
  CAR T cell: 0.0043
  Endothelial Cells: 0.0041
  Hep G2: 0.0000
  Human Preadipocyte SGBS Cells, Adipocytes: 0.0096
  Human iPSCs, Human NSCs: 0.0033
  Jurkat: 0.0000
  K562 chronic myelogenous leukemia cell: 0.0025
  T Cytotoxic: 0.0043
  T Low quality cells: 0.0039
  T Memory: 0.0046
  T Naive: 0.0045
  T Th1/Cytotoxic: 0.0046
  T Th1/Th17: 0.0049
  T Th17: 0.0049
  T Th2: 0.0046
  T Treg memory 1: 0.0050
  T Treg memory 2: 0.0047
  T Treg memory 3: 0.0040
  T Treg naive: 0.0045
  T98G: 0.0218
  THP-1 cell: 0.0046
  U87MG: 0.0218
  astrocytes: 0.0043
  cortical organoid: 0.0067
  disseminated tumour cell: 0.0000
  embryonic kidney cell: 0.0032


...

Epoch 8/10, Average Training Loss: 0.0058
Overall Validation MSE: 0.0064
MSE by Organism:
  Human Cytomegalovirus (HCMV): 0.0050
  Humans (Homo sapiens): 0.0072
  Mice (Mus musculus): 0.0022
  Zebrafish (Danio rerio): 0.0000
  human adipocytes: 0.0090
MSE by Cell Type:
  A172: 0.0198
  B-cell acute lymphoblastic leukemia (B-ALL) cell: 0.0000
  CAR T cell: 0.0040
  Endothelial Cells: 0.0040
  Hep G2: 0.0000
  Human Preadipocyte SGBS Cells, Adipocytes: 0.0090
  Human iPSCs, Human NSCs: 0.0033
  Jurkat: 0.0000
  K562 chronic myelogenous leukemia cell: 0.0025
  T Cytotoxic: 0.0042
  T Low quality cells: 0.0038
  T Memory: 0.0045
  T Naive: 0.0044
  T Th1/Cytotoxic: 0.0045
  T Th1/Th17: 0.0048
  T Th17: 0.0048
  T Th2: 0.0045
  T Treg memory 1: 0.0049
  T Treg memory 2: 0.0046
  T Treg memory 3: 0.0039
  T Treg naive: 0.0043
  T98G: 0.0196
  THP-1 cell: 0.0045
  U87MG: 0.0193
  astrocytes: 0.0043
  cortical organoid: 0.0065
  disseminated tumour cell: 0.0000
  embryonic kidney cell: 0.0032


Epoch 9/10: 100%|██████████| 4367/4367 [06:14<00:00, 11.65it/s]


Epoch 9/10, Average Training Loss: 0.0057
Overall Validation MSE: 0.0063
MSE by Organism:
  Human Cytomegalovirus (HCMV): 0.0050
  Humans (Homo sapiens): 0.0071
  Mice (Mus musculus): 0.0022
  Zebrafish (Danio rerio): 0.0000
  human adipocytes: 0.0089
MSE by Cell Type:
  A172: 0.0194
  B-cell acute lymphoblastic leukemia (B-ALL) cell: 0.0000
  CAR T cell: 0.0040
  Endothelial Cells: 0.0040
  Hep G2: 0.0000
  Human Preadipocyte SGBS Cells, Adipocytes: 0.0089
  Human iPSCs, Human NSCs: 0.0033
  Jurkat: 0.0000
  K562 chronic myelogenous leukemia cell: 0.0024
  T Cytotoxic: 0.0042
  T Low quality cells: 0.0038
  T Memory: 0.0045
  T Naive: 0.0044
  T Th1/Cytotoxic: 0.0044
  T Th1/Th17: 0.0047
  T Th17: 0.0047
  T Th2: 0.0044
  T Treg memory 1: 0.0048
  T Treg memory 2: 0.0046
  T Treg memory 3: 0.0039
  T Treg naive: 0.0043
  T98G: 0.0191
  THP-1 cell: 0.0045
  U87MG: 0.0189
  astrocytes: 0.0043
  cortical organoid: 0.0065
  disseminated tumour cell: 0.0000
  embryonic kidney cell: 0.0031


Epoch 10/10: 100%|██████████| 4367/4367 [06:16<00:00, 11.61it/s]


Epoch 10/10, Average Training Loss: 0.0057
Overall Validation MSE: 0.0063
MSE by Organism:
  Human Cytomegalovirus (HCMV): 0.0050
  Humans (Homo sapiens): 0.0071
  Mice (Mus musculus): 0.0022
  Zebrafish (Danio rerio): 0.0000
  human adipocytes: 0.0088
MSE by Cell Type:
  A172: 0.0193
  B-cell acute lymphoblastic leukemia (B-ALL) cell: 0.0000
  CAR T cell: 0.0040
  Endothelial Cells: 0.0040
  Hep G2: 0.0000
  Human Preadipocyte SGBS Cells, Adipocytes: 0.0088
  Human iPSCs, Human NSCs: 0.0033
  Jurkat: 0.0000
  K562 chronic myelogenous leukemia cell: 0.0025
  T Cytotoxic: 0.0041
  T Low quality cells: 0.0037
  T Memory: 0.0045
  T Naive: 0.0044
  T Th1/Cytotoxic: 0.0044
  T Th1/Th17: 0.0047
  T Th17: 0.0047
  T Th2: 0.0044
  T Treg memory 1: 0.0048
  T Treg memory 2: 0.0046
  T Treg memory 3: 0.0039
  T Treg naive: 0.0043
  T98G: 0.0190
  THP-1 cell: 0.0045
  U87MG: 0.0187
  astrocytes: 0.0043
  cortical organoid: 0.0064
  disseminated tumour cell: 0.0000
  embryonic kidney cell: 0.0031

In [ ]:
import os
import torch
import numpy as np
import pandas as pd
import scanpy as sc
from tqdm import tqdm
from torch.utils.data import DataLoader, TensorDataset


def infer_gene_expression(model, initial_expression, embedding, device):
    model.eval()
    with torch.no_grad():
        x0 = initial_expression.to(device)
        embedding = embedding.to(device)

        ode_func = ODEFunc(model, embedding)

        node = NeuralODE(
            ode_func,
            sensitivity='adjoint',
            solver='dopri5',
            atol=1e-4,
            rtol=1e-4
        ).to(device)

        trajectory = node.trajectory(x0, t_span=torch.linspace(0, 1, 2).to(device))
        pred_x1 = trajectory[-1]

    return pred_x1

# Load the last epoch model
save_dir = 'models/gene_expression_flow_matching'
last_epoch = max([int(f.split('_')[-1].split('.')[0]) for f in os.listdir(save_dir) if f.startswith('model_epoch_')])
model_path = os.path.join(save_dir, f'model_epoch_{last_epoch}.pt')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GeneExpressionFlowMatcher(input_dim, embedding_dim).to(device)  # Make sure to define input_dim and embedding_dim
model.load_state_dict(torch.load(model_path, map_location=device, weights_only=True))
model.eval()

# Load dataset
adata = sc.read_h5ad('/home/ubuntu/D2_all_clean_hvg_filtered.h5ad')

# Prepare data for all cells
all_data = adata.X.toarray()  # Convert to dense array if it's sparse
all_perturbations = adata.obs['perturbation_name'].tolist()

# Create embeddings for all perturbations
all_embeddings = np.array([get_embedding(p) for p in all_perturbations])

# Create dataset and dataloader
dataset = TensorDataset(
    torch.FloatTensor(all_data),
    torch.FloatTensor(all_embeddings)
)
data_loader = DataLoader(dataset, batch_size=64, shuffle=False)

# Perform inference
all_predictions = []
with torch.no_grad():
    for x0, embedding in tqdm(data_loader, desc="Processing all cells"):
        x0, embedding = x0.to(device), embedding.to(device)
        pred_x1 = infer_gene_expression(model, x0, embedding, device)
        all_predictions.append(pred_x1.cpu().numpy())

# Concatenate all predictions
all_predictions = np.concatenate(all_predictions, axis=0)

# Save predictions to adata.obsm['OT_pred']
adata.obsm['OT_pred'] = all_predictions

# Save updated adata object
adata.write_h5ad('/home/ubuntu/D2_clean_hvg_filtered_with_OT_predictions.h5ad')

# Calculate MSE for test cells
test_mask = adata.obs['condition'] == 'test'
test_pred = all_predictions[test_mask]
test_true = adata.X.toarray()[test_mask]
mse = ((test_pred - test_true) ** 2).mean(axis=1)

# Create results DataFrame
results = pd.DataFrame({
    'organism': adata.obs['organism'][test_mask],
    'cell_type': adata.obs['cell_type'][test_mask],
    'mse': mse
})

# Calculate and print overall MSE
overall_mse = mse.mean()
print(f"Overall MSE: {overall_mse:.4f}")

# Calculate and print MSE by organism
organism_mse = results.groupby('organism')['mse'].mean()
print("MSE by Organism:")
print(organism_mse)

# Calculate and print MSE by cell type
cell_type_mse = results.groupby('cell_type')['mse'].mean()
print("MSE by Cell Type:")
print(cell_type_mse)

# Save results to CSV
results.to_csv('inference_results.csv', index=False)
print("Results saved to inference_results.csv")

# Save summary statistics
with open('inference_summary.txt', 'w') as f:
    f.write(f"Overall MSE: {overall_mse:.4f}\n\n")
    f.write("MSE by Organism:\n")
    f.write(organism_mse.to_string())
    f.write("\n\nMSE by Cell Type:\n")
    f.write(cell_type_mse.to_string())
print("Summary statistics saved to inference_summary.txt")